In [1]:
import numpy as np
import matplotlib.pylab as plt
from math import *
#from sklearn.models import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn import svm
import pandas as pd
import seaborn as sns
from pylab import rcParams
from matplotlib import rc
from mpl_toolkits.mplot3d import Axes3D
import time
%matplotlib notebook

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
RANDOM_SEED = 42
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE = ['#01BEFE', '#FFDD00', '#FF7D00', '#FF006D', '#ADFF02', '#8F00FF']
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae 
from sklearn.metrics import mean_absolute_percentage_error as mape 
from sklearn.metrics import r2_score as r2 
from sklearn.metrics import explained_variance_score as evs 

In [5]:
def rmse(y_pred, y_true):
    return np.sqrt(mse(y_pred, y_true))

In [6]:
import pickle

#### April Dataset

In [7]:
df = pd.read_csv('inputs/DS1_1440.csv')

In [8]:
from processing import create_features, split_data

In [9]:
train_len = int(.6*df.shape[0])
train_data, val_data, test_data = split_data(df.wind_speed.values, train_len)

In [10]:
from lssvm import LSSVMRegression as LSSVM

In [11]:
def get_data(file_name):
    file = open(file_name, 'rb')
    data = pickle.load(file)
    file.close()
    return data

In [12]:
svmd_ebqpso_1 = get_data('svmd_ebqpso_1')

In [13]:
error_values = get_data('error_svmd_eb_lssvm_1')

In [14]:
test_preds = []

#### Get Predicted Values

In [15]:
svmds = pd.read_csv('inputs/SVMD_1.csv')

In [16]:
for i in range(len(svmd_ebqpso_1)):
    params = svmd_ebqpso_1[i][0]
    train_data_i, _, test_data_i = split_data(svmds.iloc[:, i].values, train_len)
    x_train_i, y_train_i = create_features(train_data_i, params[-1])
    x_test_i, y_test_i = create_features(test_data_i, params[-1])
    reg = LSSVM(gamma=params[0], sigma=params[1])
    reg.fit(x_train_i, y_train_i)
    test_preds.append(reg.predict(x_test_i))

In [21]:
# Get the optimum values of window sizes for each modes
window_sizes = np.array([svmd_ebqpso_1[i][0][-1] for i in svmd_ebqpso_1.keys()])

In [22]:
max_window_size = np.max(window_sizes)

In [23]:
adjusted_test_preds = np.array([
    list(test_preds[i][(max_window_size-window_sizes[i]):]) for i in range(len(window_sizes))
])

In [24]:
total_modes_preds = adjusted_test_preds.sum(axis=0)

In [25]:
total_modes_preds.shape

(276,)

In [29]:
error_values_1 = error_values[max_window_size-5:]

In [30]:
svmd_ebqpso_lssvm_preds = total_modes_preds + error_values_1

In [33]:
svmd_ebqpso_lssvm_preds.shape

(276,)

In [35]:
_, y_test = create_features(test_data, max_window_size)

In [45]:
mae(y_test, svmd_ebqpso_lssvm_preds)

0.512030131411579

##### CNN Model

In [56]:
cnn_preds = get_data('pred_cnn')

In [57]:
len(cnn_preds)

280

In [58]:
cnn_preds = cnn_preds[max_window_size-5:]

In [59]:
len(cnn_preds)

273

In [55]:
svmd_ebqpso_lssvm_preds.shape

(276,)